In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
import gc
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reduced_df = pd.read_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_df.csv')
reduced_df

/tmp/ipykernel_155484/1069376804.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_df = pd.read_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_df.csv')


,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,memory,code_size,accuracy,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,3108.0,250,1/1,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,3080.0,232,1/1,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,3096.0,182,1/1,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,2588.0,129,1/1,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,3080.0,180,1/1,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,6818156,2073,s921588169,p04050,u748871552,1470008541,C++,C++11 (GCC 4.8.1),cpp,Accepted,...,256.0,1080,NaN,1,s921588169,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,"#include <bits/stdc++.h>\n\n#define REP(i,n) f...",Here is the code summary format for the given ...
499996,6818157,2074,s191865152,p04050,u955723966,1469987618,C++,C++14 (GCC 5.4.1),cpp,Accepted,...,256.0,1066,NaN,1,s191865152,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\n\nusing namespace st...,Here is the code summary format for the given ...
499997,6818166,2083,s564884250,p04050,u572012381,1469778717,C++,C++14 (GCC 5.4.1),cpp,Accepted,...,256.0,1577,NaN,1,s564884250,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\nusing namespace std;...,Here is the code summary format for the given ...
499998,6818168,2085,s162996393,p04050,u657011188,1469479168,C++,C++14 (GCC 5.4.1),cpp,Accepted,...,256.0,1021,NaN,1,s162996393,p04050,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <bits/stdc++.h>\nusing namespace std;...,Here is the code summary format for the given ...


In [3]:
chunks = np.array_split(reduced_df, 10)

# Optionally assign them to named variables if you'd like:
df1, df2, df3, df4, df5,df6,df7,df8,df9,df10= chunks

df1

/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,Unnamed: 0,index,submission_id,problem_id,user_id,date,language,original_language,filename_ext,status,...,memory,code_size,accuracy,Correct/incorrect,submission_idstr,problem_idstr,test_path,sum_path,code,summary
0,4850,8415,s667847559,p00000,u972675635,1530881789,C++,C++11,cpp,Accepted,...,3108.0,250,1/1,1,s667847559,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
1,4851,8416,s160425098,p00000,u642752018,1530897493,C++,C++,cpp,Accepted,...,3080.0,232,1/1,1,s160425098,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<iostream>\nusing namespace std;\n\nin...,Here's the code summary for the given problem ...
2,4852,8417,s682094990,p00000,u313584000,1531051141,C++,C++,cpp,Accepted,...,3096.0,182,1/1,1,s682094990,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\nint...,Here's the code summary for the given problem ...
3,4854,8419,s068595017,p00000,u011621222,1531399592,C++,C++,cpp,Accepted,...,2588.0,129,1/1,1,s068595017,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<cstdio>\nint main(){\n\tfor(int i = 1...,Here's the code summary for the given problem ...
4,4857,8422,s224013242,p00000,u937856796,1531468490,C++,C++,cpp,Accepted,...,3080.0,180,1/1,1,s224013242,p00000,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\nusing namespace std;\n\ni...,Here's the code summary for the given problem ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,335037,3584,s223598643,p02277,u499717478,1568183643,C++,C++,cpp,Accepted,...,4924.0,1798,10/10,1,s223598643,p02277,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include<stdio.h>\n#include<algorithm>\nusing ...,Here's the code summary for the Quick Sort pro...
49996,335038,3585,s311459153,p02277,u512700093,1568017908,C++,C++,cpp,Accepted,...,5076.0,1774,10/10,1,s311459153,p02277,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <stdio.h>\n\n#define MAX 100000\n#def...,Here's the code summary for the Quick Sort pro...
49997,335039,3586,s300820929,p02277,u789814124,1567984828,C++,C++,cpp,Accepted,...,5436.0,1878,10/10,1,s300820929,p02277,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n#include <algorithm>\n#in...,Here's the code summary for the Quick Sort pro...
49998,335040,3587,s471517339,p02277,u437549542,1567252191,C++,C++,cpp,Accepted,...,5732.0,2361,10/10,1,s471517339,p02277,/home/profniggastein/PycharmProjects/ReducedDa...,/home/profniggastein/PycharmProjects/ReducedDa...,#include <iostream>\n#include <vector>\nusing ...,Here's the code summary for the Quick Sort pro...


In [4]:
client = OpenAI(api_key="sk-9a18103e17be4329a06b65b1520c5d9b", base_url="https://api.deepseek.com")


def GenerateSummary(text):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": """You are are an assistant/tutor helping users understand various code snippets by giving a general summary and as much as poosible give them  in a general summary form for specifically c and c++ code like this;Code Summary Format

    1. Function/Method Name:
    process_data

    2. Purpose:
    Briefly describe what the code is intended to do.
    E.g., Processes and cleans input data by removing null values and standardizing formats.

    3. Inputs:

    dataframe (pandas.DataFrame): The input dataset containing raw values.

    columns (List[str]): Specific columns to be processed.


    4. Outputs:

    cleaned_dataframe (pandas.DataFrame): A cleaned version of the input dataset.


    5. Key Operations/Logic:

    Drops rows with null values in specified columns

    Converts column names to lowercase

    Normalizes date formats


    6. Dependencies:

    pandas

    datetime


    7. Edge Cases Handled:

    Returns an empty DataFrame if all rows contain nulls

    Skips processing if columns is empty


    8. Example Usage (Optional):

    clean_df = process_data(df, ["date", "price", "location"])"""},
            {"role": "user", "content": f"{text}"},
        ],
        stream=False
    )
    return response



In [5]:

def calculate_bleu(reference, candidate):
    reference = reference.split()
    candidate = candidate.split()
    return sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)




In [6]:
  df = df1.copy()
df['GenSummary'] = df['code'].apply(
    lambda x: GenerateSummary(x).choices[0].message.content if pd.notnull(x) else None
)
df.to_csv('/home/profniggastein/PycharmProjects/ReducedDataset/Project_CodeNet/Reduced_sample_df1.csv', index=False)

df

APIConnectionError: Connection error.

In [ ]:
df['bleu_score'] = df.apply(
    lambda row: calculate_bleu(row['GenSummary'], row['summary']) if pd.notnull(row['summary']) and pd.notnull(
        row['GenSummary']) else None,
    axis=1
)
df['BERT_score'] = df.apply(
    lambda row: score([row["GenSummary"]], [row["summary"]], model_type="microsoft/codebert-base")[2][0].item(),
    axis=1
)
df
